# DSA4263 E-Commerce Fraud Analysis

## Import Packages

In [ ]:
# Uncomment if needed to install the packages
# pip install pandas numpy

In [12]:
import pandas as pd
import numpy as np

## Data Loading

In [104]:
# Uncomment this cell when submitting

# df_fraud = pd.read_csv("data/Fraud_Data.csv")
# df_ip_to_country = pd.read_csv("data/IpAddress_to_Country.csv")

In [88]:
# Delete this cell when submitting

from google.colab import drive
drive.mount("/content/gdrive")
df_fraud = pd.read_csv("gdrive/MyDrive/Fraud_Data.csv")
df_ip_to_country = pd.read_csv("gdrive/MyDrive/IpAddress_to_Country.csv")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Data Preprocessing

In [93]:
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [90]:
df_fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         151112 non-null  int64  
 1   signup_time     151112 non-null  object 
 2   purchase_time   151112 non-null  object 
 3   purchase_value  151112 non-null  int64  
 4   device_id       151112 non-null  object 
 5   source          151112 non-null  object 
 6   browser         151112 non-null  object 
 7   sex             151112 non-null  object 
 8   age             151112 non-null  int64  
 9   ip_address      151112 non-null  float64
 10  class           151112 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 12.7+ MB


In [94]:
df_ip_to_country.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [91]:
df_ip_to_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138846 entries, 0 to 138845
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   lower_bound_ip_address  138846 non-null  float64
 1   upper_bound_ip_address  138846 non-null  int64  
 2   country                 138846 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 3.2+ MB


In [101]:
# Checking for null values and duplicates in Fraud_Data.csv

print(df_fraud.isna().sum())
print("--------------------")
print(df_fraud.duplicated().sum())

user_id           0
signup_time       0
purchase_time     0
purchase_value    0
device_id         0
source            0
browser           0
sex               0
age               0
ip_address        0
class             0
dtype: int64
--------------------
0


In [103]:
# Checking for null values and duplicates in IpAddress_to_country.csv

print(df_ip_to_country.isna().sum())
print("--------------------")
print(df_ip_to_country.duplicated().sum())

lower_bound_ip_address    0
upper_bound_ip_address    0
country                   0
dtype: int64
--------------------
0


There are no null values and duplicate entries in both datasets.

In [107]:
# Checking for duplicates in `user_id` and `device_id` in Fraud_Data.csv

print("No. of duplicates in user_id:", df_fraud['user_id'].duplicated().sum())
print("No. of duplicates in device_id:", df_fraud['device_id'].duplicated().sum())

No. of duplicates in user_id: 0
No. of duplicates in device_id: 13156


While `user_id` are unique, there are repeated instances of `device_id`, indicating that multiple accounts have been created using the same device. Based on this observation, we will drop the `user_id` column from our model and retain the `device_id` column.

In [120]:
df_clean = df_fraud.drop('user_id', axis=1)

In [123]:
# Merge the datasets based on Ip Address

def find_country(ip_address, df_ip_ranges):
  country = df_ip_ranges[
      (df_ip_ranges['lower_bound_ip_address'] <= ip_address) &
      (df_ip_ranges['upper_bound_ip_address'] >= ip_address)
  ]['country'].values

  return country[0] if len(country) > 0 else 'Unknown'

country_values = df_clean['ip_address'].apply(find_country, df_ip_ranges=df_ip_to_country)

# Insert `country` column to the second last position of the columns
df_clean.insert(loc = len(df_clean.columns) - 1,
                column = 'country', value = country_values)

# Remove the Ip Address
df_clean.drop('ip_address', axis=1, inplace=True)

In [130]:
# Finding the number of rows with unknown country

print("No. of unknown country:", (df_clean['country'] == 'Unknown').sum())

No. of unknown country: 21966


In [131]:
# Removing rows with unkown country
df_clean = df_clean[df_clean['country'] != 'Unknown']

In [137]:
df_clean.head()

,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,country,class
0,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,Japan,0
1,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,United States,0
2,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,United States,1
4,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,United States,0
5,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,Canada,0
